---
PBMCs Example:

Getting MLE/Bayes? probability estimates for the 4 cell types, CD4, CD8, CD14, and NK cells.

These PEs are the dataset barcodes -- basically giving a distribution in each of the different cell types for all the genes in the cell?


In [1]:
# Enter commands in R -- no need to run if they are already installed
#install.packages('curl', repos = c('https://jeroen.r-universe.dev', 'https://cloud.r-project.org'))
#install.packages("httr2")
# on the mac make sure you have nlopt installed needed by the sads package
# which is needed by Seurat
# brew install nlopt and then install the r package
#install.packages("nloptr")
#install.packages("Rcpp")
# Rcpp is needed by the GUILDS package 
#install.packages("GUILDS")
#install.packages("sads", repos = "https://cloud.r-project.org/")
#install.packages('Seurat')

In [2]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("UCSC.utils")
BiocManager::install("KEGGREST")
BiocManager::install("GenomeInfoDb")
BiocManager::install("AnnotationDbi")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("BiocParallel")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'UCSC.utils'”
Old packages: 'bit', 'cpp11', 'RSQLite'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'KEGGREST'”
Old packages: 'bit', 'cpp11', 'RSQLite'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "B

In [3]:
library(sads)

Loading required package: stats4

Loading required package: bbmle



In [4]:
library(org.Hs.eg.db)
library(Seurat)

Loading required package: AnnotationDbi

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading required package: S4Vectors


Attaching package: ‘S4Vector

In [5]:
getwd()

[1] "/Users/annedeslattesmays/Scitechcon Dropbox/Anne DeslattesMays/projects/scRNAseq-cell-type/docs/ipynb"

In [7]:
setwd("../../original")

In [8]:
source ("cell_type_identification4_clean.R")

We start by reading in our training data, which is downloaded from 10X Genomics: CD4, CD8, CD14, and NK cells. For the purposes of this example, we will withhold 100 cells of each to serve as test data. After loading in the data, we convert the gene symbols to Ensembl IDs (required).

In [9]:
getwd()
setwd("..")
getwd()
system("mkdir data")
setwd("data")
getwd()

[1] "/Users/annedeslattesmays/Scitechcon Dropbox/Anne DeslattesMays/projects/scRNAseq-cell-type/original"

[1] "/Users/annedeslattesmays/Scitechcon Dropbox/Anne DeslattesMays/projects/scRNAseq-cell-type"

[1] "/Users/annedeslattesmays/Scitechcon Dropbox/Anne DeslattesMays/projects/scRNAseq-cell-type/data"

the data for these next steps are found at the https sites and all were used with the filtered matrices.  The steps are:

1. wget the location
2. tar xvfz to unpack the data
3. rename the location to match the code below 


In [10]:
## CD4: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cd4_t_helper?  
system ("wget https://cf.10xgenomics.com/samples/cell-exp/1.1.0/cd4_t_helper/cd4_t_helper_filtered_gene_bc_matrices.tar.gz")
system ("tar xvfz cd4_t_helper_filtered_gene_bc_matrices.tar.gz")
system ("mv filtered_matrices_mex cd4_singlecell")

In [11]:
## CD8: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cytotoxic_t
system ("wget https://cf.10xgenomics.com/samples/cell-exp/1.1.0/cytotoxic_t/cytotoxic_t_filtered_gene_bc_matrices.tar.gz")
system ("tar xvfz cytotoxic_t_filtered_gene_bc_matrices.tar.gz")
system ("mv filtered_matrices_mex filtered_matrices_cd8")

In [12]:
## CD14: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cd14_monocytes
system ("wget https://cf.10xgenomics.com/samples/cell-exp/1.1.0/cd14_monocytes/cd14_monocytes_filtered_gene_bc_matrices.tar.gz")
system ("tar xvfz cd14_monocytes_filtered_gene_bc_matrices.tar.gz")
system ("mv filtered_matrices_mex filtered_matrices_cd14")

In [13]:
## NK: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cd56_nk
system ("wget https://cf.10xgenomics.com/samples/cell-exp/1.1.0/cd56_nk/cd56_nk_filtered_gene_bc_matrices.tar.gz")
system ("tar xvfz cd56_nk_filtered_gene_bc_matrices.tar.gz")
system ("mv filtered_matrices_mex filtered_matrices_nk")

In [14]:
## clean up tar.gz files
system ("rm *.gz")


In [15]:
getwd()

[1] "/Users/annedeslattesmays/Scitechcon Dropbox/Anne DeslattesMays/projects/scRNAseq-cell-type/data"

In [16]:
## mv back to original directory where R code is
setwd("../original")


In [17]:
getwd()

[1] "/Users/annedeslattesmays/Scitechcon Dropbox/Anne DeslattesMays/projects/scRNAseq-cell-type/original"

In [18]:
set.seed(6619)

### Preparation for symbol->ENSEMBL conversion
Hs_symbol <- org.Hs.egSYMBOL
mapped_Hs_genes.symbol <- mappedkeys(Hs_symbol)
Hs_symbol.df <- as.data.frame(Hs_symbol[mapped_Hs_genes.symbol])
Hs_ensembl <- org.Hs.egENSEMBL
mapped_Hs_genes.ensembl <- mappedkeys(Hs_ensembl)
Hs_ensembl.df <- as.data.frame(Hs_ensembl[mapped_Hs_genes.ensembl])
Hs_mapping <- merge(Hs_symbol.df,Hs_ensembl.df)



In [19]:
## CD4: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cd4_t_helper? 
cd4_facs.data <- Read10X(data.dir = "../data/cd4_singlecell/hg19/") 
cd4_facs.data <- as.matrix(cd4_facs.data)
rownames(cd4_facs.data) <- Hs_mapping$ensembl_id[match(rownames(cd4_facs.data),
                                                       Hs_mapping$symbol)]
cd4_facs.data <- cd4_facs.data[!is.na(rownames(cd4_facs.data)),]
cd4_facs.data <- na.omit(cd4_facs.data)
cd4.test <- cd4_facs.data[,1:100] # CD4 withheld cells
cd4_facs.data <- cd4_facs.data[,101:5101] # CD4 training cells

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”


In [20]:
## CD8: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cytotoxic_t
cd8_facs.data <- Read10X(data.dir = '../data/filtered_matrices_cd8/hg19/')
cd8_facs.data <- as.matrix(cd8_facs.data) 
rownames(cd8_facs.data) <- Hs_mapping$ensembl_id[match(rownames(cd8_facs.data),
                                                       Hs_mapping$symbol)]
cd8_facs.data <- cd8_facs.data[!is.na(rownames(cd8_facs.data)),]
cd8.test <- cd8_facs.data[,1:100] # CD8 withheld cells
cd8_facs.data <- cd8_facs.data[,101:5101] # CD8 training cells

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.5 GiB”


In [21]:
## CD14: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cd14_monocytes
cd14_facs.data <- Read10X(data.dir = '../data/filtered_matrices_cd14/hg19/')
cd14_facs.data <- as.matrix(cd14_facs.data)
rownames(cd14_facs.data) <- Hs_mapping$ensembl_id[match(rownames(cd14_facs.data),
                                                        Hs_mapping$symbol)]
cd14_facs.data <- cd14_facs.data[!is.na(rownames(cd14_facs.data)),]
cd14.test <- cd14_facs.data[,1:100] # CD14 withheld cells
cd14_facs.data <- cd14_facs.data[,101:ncol(cd14_facs.data)] # CD14 training cells

In [22]:
## NK: https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/cd56_nk
nk_facs.data <- Read10X(data.dir = '../data/filtered_matrices_nk/hg19/')
nk_facs.data <- as.matrix(nk_facs.data)
rownames(nk_facs.data) <- Hs_mapping$ensembl_id[match(rownames(nk_facs.data),
                                                      Hs_mapping$symbol)]
nk_facs.data <- nk_facs.data[!is.na(rownames(nk_facs.data)),]
nk.test <- nk_facs.data[,1:100] # NK withheld cells
nk_facs.data <- nk_facs.data[,101:5101] # NK training cells

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.0 GiB”


Next, we fit our model to these training data. We combine the training data into a single matrix and call the `trainAllReference` function on this matrix, alongside a vector of cell-type labels. By default, this function will only consider the subset of discriminative genes needed for classification. For now, to illustrate the uses of our method additional to classification, we will consider all genes by setting `discrim_only=FALSE`. (Note that regardless of whether this parameter is set to `TRUE`, as it is by default, or `FALSE`, the classification function will still only use the subset of discriminative genes; setting this parameter to `TRUE` just has the effect of speeding up the training process by only fitting to that set of genes.)

In [23]:
## Input a single matrix with a vector of labels 
common_pbmcs_genes <- Reduce(intersect,list(rownames(cd4_facs.data),
                                            rownames(cd14_facs.data),
                                            rownames(cd8_facs.data),
                                            rownames(nk_facs.data)))
pbmcs_reference <- as.matrix(cbind(cd4_facs.data[common_pbmcs_genes,],
                                   cd14_facs.data[common_pbmcs_genes,],
                                   cd8_facs.data[common_pbmcs_genes,],
                                   nk_facs.data[common_pbmcs_genes,]))
pbmcs_reference_labels <- c(rep('CD4',dim(cd4_facs.data)[2]),rep('CD14',dim(cd14_facs.data)[2]),
                            rep('CD8',dim(cd8_facs.data)[2]),rep('NK',dim(nk_facs.data)[2]))
pbmcs.d <- trainAllReference(pbmcs_reference,pbmcs_reference_labels,discrim_only=FALSE)

The list `pbmcs.d` contains one table for each inputted cell-type, which indicates each gene's empirical rate, probability of belonging to the off-low latent state, and probability of belonging to the off-high latent state. This is used as input into the `classifyTarget` function. However, if our goal is to examine each cell-type's barcode, we could pass this list into the `getBarcode` function, which will present the probability that each gene is on in each cell-type. This could then be used beyond the cell-type annotation context to study gene expression within a single cell-type, to compare genes across cell-types, and to identify markers. 

As an example, we show below how we can find the barcodes for each cell-type, then identify genes with a high probability of being on in NK cells, but a low probability of being on in the others (i.e. potential markers for this cell-type). Finally, we show how we can identify markers for NK cells compared to global patterns of gene expression, rather than the other cell-types under consideration here.

In [24]:
## Obtain barcodes
barcodes <- getBarcode(pbmcs.d)

## Identify genes likely to be on in NK cells, but likely to be off in the others
findMarkers('NK',barcodes,thresh_up=0.95,thresh_below=0.05)

## Identify genes likely to be on in NK cells, and not in global patterns 
findMarkers('NK',barcodes,thresh_up=0.99,thresh_below=0.01,relative=F)

,CD4,CD14,CD8,NK
ENSG00000143184,9.081278e-06,4.207157e-05,9.364434e-03,0.9514225
ENSG00000117560,1.706751e-06,7.434783e-07,7.677824e-05,0.9567601
ENSG00000115607,3.217127e-11,9.485573e-11,4.448123e-05,0.9663229
ENSG00000180871,1.819799e-10,1.760079e-04,4.004241e-11,0.9981620
ENSG00000171596,2.231033e-11,2.469601e-09,1.743697e-08,0.9543838
ENSG00000086200,5.870675e-03,3.012470e-03,1.469428e-03,0.9873570
ENSG00000104343,1.407836e-05,5.085064e-05,2.060410e-03,0.9999573
ENSG00000150045,1.326175e-04,3.719724e-03,3.599665e-02,0.9979407
ENSG00000240403,6.416552e-10,1.023004e-11,4.178196e-08,0.9775072


,NK
ENSG00000158062,0.9999977
ENSG00000126088,1.0000000
ENSG00000173660,1.0000000
ENSG00000162402,0.9994690
ENSG00000162607,1.0000000
ENSG00000077254,1.0000000
ENSG00000143569,1.0000000
ENSG00000160714,1.0000000
ENSG00000143222,1.0000000
ENSG00000173960,1.0000000


Finally, we can use the original object `pbmcs.d` to annotate our withheld test cells, which should be placed in a single matrix. If we want the possibility of detecting cells that do not belong to any of the cell-types in the reference, we can set `other=T`. Further, we can specify `return.probs=T` if we want a matrix of probabilistic assignments rather than just a vector of highest-probability cell-type labels. We show both below:

In [25]:
## Put withheld test cells in one matrix
pbmcs_withheld <- as.matrix(cbind(cd4.test[common_pbmcs_genes,],
                                  cd14.test[common_pbmcs_genes,],
                                    cd8.test[common_pbmcs_genes,],
                                  nk.test[common_pbmcs_genes,]))
true_labels <- c(rep('CD4',100),rep('CD14',100),rep('CD8',100),rep('NK',100))

## Annotate!
annotation <- classifyTarget(pbmcs_withheld,pbmcs.d,other=T)
table(annotation,true_labels)

## Return probabilistic assignments instead
probabilistic_annotation <- classifyTarget(pbmcs_withheld,pbmcs.d,other=T,
                                           return.probs=T)
head(round(probabilistic_annotation,2))

          true_labels
annotation CD14 CD4 CD8 NK
      CD14   99   0   0  0
      CD4     0  94   4  0
      CD8     1   6  96  1
      NK      0   0   0 99

CD4,CD14,CD8,NK,other
1.00,0,0.00,0,0
0.98,0,0.02,0,0
0.94,0,0.06,0,0
0.50,0,0.50,0,0
0.14,0,0.86,0,0
0.98,0,0.02,0,0


In [26]:
dim(barcodes) 
head(barcodes,10)


[1] 18573     4

,CD4,CD14,CD8,NK
ENSG00000237613,2.064321e-16,2.255141e-16,1.925543e-16,1.656661e-16
ENSG00000186092,1.942283e-14,1.897441e-14,1.993978e-14,1.989468e-14
ENSG00000284662,1.943844e-14,1.915829e-14,1.997881e-14,2.009157e-14
ENSG00000177757,8.556509e-10,7.015680e-10,9.125983e-10,9.090474e-10
ENSG00000230368,2.784804e-13,2.089162e-13,1.534398e-13,6.531997e-13
ENSG00000187634,5.668630e-09,4.952028e-06,1.043756e-10,1.144851e-10
ENSG00000188976,2.131254e-08,2.690179e-08,9.956885e-09,1.427331e-08
ENSG00000187961,5.764123e-08,4.637252e-05,4.626105e-07,4.097974e-06
ENSG00000187583,3.200113e-08,3.583235e-08,5.886827e-10,1.100284e-11
ENSG00000188290,6.420500e-04,6.712186e-01,8.765021e-04,1.310125e-03
